In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import time
from tqdm import tqdm  # For progress bar
from matplotlib import pyplot as plt
import plotly.graph_objects as go

## Test with video

### Setup serial communication

In [ ]:
import serial

# Open serial connection to Arduino
arduino = serial.Serial(port='COM12', baudrate=115200, timeout=1)  # Adjust COM port as needed

In [ ]:
import serial
import threading
import time
from IPython.display import display
import ipywidgets as widgets

# Create an output widget
output_widget = widgets.Output()
display(output_widget)

def serial_monitor():
    """ Continuously reads from Arduino and updates output widget in Jupyter. """
    while True:
        if arduino.in_waiting > 0:
            line = arduino.readline().decode('utf-8').strip()
            with output_widget:
                output_widget.clear_output(wait=True)  # Clears old messages dynamically
                print("Arduino says:", line)
        time.sleep(0.1)  # Prevent excessive CPU usage

# Start the serial monitor in a background thread
thread = threading.Thread(target=serial_monitor, daemon=True)
thread.start()

## Process video as frames with mediapipe

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import os
import time
from tqdm import tqdm  # For progress bar
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import tensorflow as tf
import numpy as np

In [ ]:
final_mlp = tf.keras.models.load_model('final_model.keras')

In [ ]:
scaling_factor = [160.0, 160.0, 160.0, 160.0, 100.0]

In [ ]:
# Define output paths for the original and annotated videos
original_video_path = 'cam_video.mp4'
annotated_video_path = 'cam_video_annotated.mp4'

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hand_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Start capturing from the laptop camera
cap = cv2.VideoCapture(0)
fps = 2  # Adjust FPS as needed
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define codecs and VideoWriter objects for both videos
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_original = cv2.VideoWriter(original_video_path, fourcc, fps, (width, height))
out_annotated = cv2.VideoWriter(annotated_video_path, fourcc, fps, (width, height))

# Set capture duration (in seconds) or None to capture until user stops
capture_duration = None  # Set this to a specific duration (e.g., 10) for timed capture or None for manual stop

# Start time for timed recording if duration is specified
start_time = time.time() if capture_duration is not None else None

# Display instructions for manual stop if no duration is set
if capture_duration is None:
    print("Press 'q' to stop the camera recording.")

In [ ]:
# Continuous processing loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Save the original frame
    out_original.write(frame)

    # Convert the frame to RGB for MediaPipe processing
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect hands
    result = hand_detector.process(frame_rgb)
    
    # Draw hand landmarks if hands are detected
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
        if len(result.multi_handedness) == 1:
            is_left = result.multi_handedness[0].classification[0].label == "Left"
            world_keypoints = [[i.x,i.y,i.z] for i in result.multi_hand_world_landmarks[0].landmark]   
            # Construct the row
            print(f"Is hand left? {is_left}")
            if is_left:
                X_test = np.array(world_keypoints).flatten()

                X_modified = X_test[3:]  # Shape becomes (a, b-3)
                c = (X_modified.shape[0]) // 3  # Compute c
                X_test_3d = X_modified.reshape(1,c, 3)

                y_pred = final_mlp.predict(X_test_3d)
                motor_angles = (y_pred*scaling_factor).round(1)

                # Send angles to Arduino as comma-separated string
                motor_angles_str = ",".join(map(str, motor_angles[0])) + "\n"
                print(motor_angles_str)
                arduino.write((motor_angles_str).encode())
                print("Sent motor angles:", motor_angles_str)
                
    # Display the annotated frame only
    cv2.imshow("Annotated Camera Feed", frame)

    # Save the annotated frame
    out_annotated.write(frame)

    # Check if 'q' is pressed for manual stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Manual stop detected. Ending recording.")
        
        # Send reset command to Arduino
        arduino.write("reset fingers\n".encode())
        print("Sent reset command to Arduino: 'reset fingers'")
        break


    # Check if the specified duration has passed for timed stop
    if capture_duration is not None and (time.time() - start_time) >= capture_duration:
        print(f"Timed capture complete after {capture_duration} seconds.")
        break

# Release resources
cap.release()
out_original.release()
out_annotated.release()
hand_detector.close()
cv2.destroyAllWindows()

# Output save paths
print(f"Original video saved as {original_video_path}")
print(f"Annotated video saved as {annotated_video_path}")

In [ ]:
arduino.write("reset fingers\n".encode())